In [1]:
# load data
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

df = pd.read_csv('../tracks.csv')
features = df.drop(columns=[col for col in df.columns if col.startswith('label_')])
features = features.drop(columns=['track_id', 'track_name', 'track_artist']) # columns not useful. Maybe use NLP for track_name/track_artist?
features

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,keymode_10_1,keymode_10_0,keymode_3_1,keymode_3_0,keymode_2_1,keymode_2_0,keymode_7_1,keymode_7_0,keymode_0_1,keymode_0_0
0,0.642049,1.201614,1.367123,-0.481362,-0.333898,-0.377953,-0.809230,0.031908,0.042927,-0.518874,...,0,0,0,0,0,0,0,0,0,0
1,0.490412,0.643317,0.585766,-0.688642,-0.468670,-0.359177,1.081061,0.782522,-0.777198,-1.056268,...,0,0,0,0,0,0,0,0,0,0
2,0.138889,1.284529,1.100090,-0.324422,-0.436799,-0.377849,-0.519562,0.439384,0.116227,-0.822017,...,0,0,0,0,0,0,0,0,0,0
3,0.435271,1.279002,0.984309,-0.050024,-0.667642,-0.377911,0.089582,-1.001795,0.039953,-0.947750,...,0,0,0,0,0,0,1,0,0,0
4,-0.033426,0.742815,0.685151,-0.702460,-0.432701,-0.377953,-0.692585,0.919777,0.115037,-0.614172,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32828,-1.563583,1.234780,1.641519,-0.132936,-0.449547,-0.377953,-0.799509,-1.289173,0.270929,-0.358076,...,0,0,0,0,1,0,0,0,0,0
32829,-0.915679,0.483014,0.755423,-0.642251,-0.790531,-0.358910,1.197705,-0.474221,0.266134,2.127922,...,0,0,0,0,0,0,0,0,1,0
32830,-0.867431,0.676483,0.609190,-0.582041,-0.306579,-0.377948,-0.260352,-0.319809,0.264201,-0.262193,...,0,0,0,0,0,0,0,0,0,0
32831,-0.198848,1.046839,1.123849,0.019069,-0.762256,0.188438,0.990337,-0.868829,0.264908,2.367121,...,0,0,0,0,1,0,0,0,0,0


First, training off of labels as only the main genre, ignoring subgenre labels.

In [2]:
labels_main_only = df[[col for col in df.columns if col.startswith('label_')]]
labels_main_only = labels_main_only.idxmax(axis=1).apply(lambda x: x.replace('label_', '').split('_')[0])
labels_main_only

0        pop
1        pop
2        pop
3        pop
4        pop
        ... 
32828    edm
32829    edm
32830    edm
32831    edm
32832    edm
Length: 32833, dtype: object

In [3]:
X_train, X_test, y_train, y_test = train_test_split(features, labels_main_only, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Step 5: Make predictions on the test set
y_pred = rf.predict(X_test)

print("Accuracy Score: ", accuracy_score(y_test, y_pred))
print("\nClassification Report for each label:\n", classification_report(y_test, y_pred))

Accuracy Score:  0.5477386934673367

Classification Report for each label:
               precision    recall  f1-score   support

         edm       0.64      0.69      0.67      1218
       latin       0.49      0.40      0.44      1033
         pop       0.36      0.32      0.34      1081
         r&b       0.46      0.47      0.47      1031
         rap       0.58      0.64      0.61      1168
        rock       0.67      0.74      0.71      1036

    accuracy                           0.55      6567
   macro avg       0.54      0.54      0.54      6567
weighted avg       0.54      0.55      0.54      6567



The accuracy score of the base model with main genre only is
```
Accuracy Score:  0.5523069894929191
```
Then, training the Random Forest model off of genre+subgenre labels combined into one label.

In [4]:
labels_main_and_sub = df[[col for col in df.columns if col.startswith('label_')]]
labels_main_and_sub = labels_main_and_sub.idxmax(axis=1).apply(lambda x: x.replace('label_', ''))
labels_main_and_sub

0                        pop_dance pop
1                        pop_dance pop
2                        pop_dance pop
3                        pop_dance pop
4                        pop_dance pop
                     ...              
32828    edm_progressive electro house
32829    edm_progressive electro house
32830    edm_progressive electro house
32831    edm_progressive electro house
32832    edm_progressive electro house
Length: 32833, dtype: object

In [5]:
X_train, X_test, y_train, y_test = train_test_split(features, labels_main_and_sub, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf.predict(X_test)

print("Accuracy Score: ", accuracy_score(y_test, y_pred))
print("\nClassification Report for each label:\n", classification_report(y_test, y_pred))

Accuracy Score:  0.24714481498401097

Classification Report for each label:
                                precision    recall  f1-score   support

                 edm_big room       0.32      0.31      0.31       248
            edm_electro house       0.32      0.38      0.34       309
                  edm_pop edm       0.12      0.09      0.10       325
edm_progressive electro house       0.29      0.41      0.34       336
          latin_latin hip hop       0.15      0.14      0.14       315
              latin_latin pop       0.17      0.11      0.13       281
              latin_reggaeton       0.30      0.32      0.31       187
               latin_tropical       0.21      0.22      0.22       250
                pop_dance pop       0.11      0.11      0.11       236
               pop_electropop       0.14      0.08      0.10       299
          pop_indie poptimism       0.18      0.20      0.19       330
            pop_post-teen pop       0.07      0.05      0.06       216

For the model with genre+subgenre labels, the accuracy score is
```
Accuracy Score:  0.25140855794122124
```

Tuning:
Looking for hyperparameters via RandomizedSearchCV, first with only main genres:

In [ ]:
# back to the main genre labels only
X_train, X_test, y_train, y_test = train_test_split(features, labels_main_only, test_size=0.2, random_state=42)

param_dist = {
    'n_estimators': [int(x) for x in np.linspace(5, 700, num=20)],
    'max_depth': [None, 20, 30, 50, 75],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'sqrt', 'log2']
}

rf = RandomForestClassifier(random_state=42, class_weight='balanced')
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist,
                                   n_iter=20, cv=5, n_jobs=-1, random_state=42, verbose=1)
random_search.fit(X_train, y_train)
best_rf = random_search.best_estimator_

y_pred = best_rf.predict(X_test)
print("Best Parameters:", random_search.best_params_)
print("Accuracy Score: ", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Took 14m54s to run, producing:
```
Best Parameters: {'n_estimators': 553, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 30}
Accuracy Score:  0.5550479671082686
```
Not much of an increase. Now to try with subgenres:



In [ ]:
# back to the main genre labels only
X_train, X_test, y_train, y_test = train_test_split(features, labels_main_and_sub, test_size=0.2, random_state=42)

param_dist = {
    'n_estimators': [int(x) for x in np.linspace(5, 700, num=5)],
    'max_depth': [None, 30, 75],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

rf = RandomForestClassifier(random_state=42, class_weight='balanced')
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist,
                                   n_iter=15, cv=5, n_jobs=-1, random_state=42, verbose=1)
random_search.fit(X_train, y_train)
best_rf = random_search.best_estimator_

y_pred = best_rf.predict(X_test)
print("Best Parameters:", random_search.best_params_)
print("Accuracy Score: ", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

```
Best Parameters: {'n_estimators': 700, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': None}
Accuracy Score:  0.26237246840261913
```
Again, not a huge increase. This seems to be the limit for Random Forest models.